# Math 134A Week 2

In [66]:
import math
from scipy.optimize import fsolve
import numpy_financial as npf
import pandas as pd

# Computing the present value of a cash flow with interest rate r. 
def PV(r, cf, continuous_compounding = False, dt = None): 
    if dt is None: dt = range(len(cf))
    pv = 0
    if not continuous_compounding:
        for k in range(0, len(cf)):
            pv += cf[k] / (1+r)**dt[k]
    else:
        for k in range(0, len(cf)):
            pv += cf[k] * math.exp(-r * dt[k])
    return pv

# This will return the future value of a cash flow at time T. If T is not specified, then we take T as the time that we receive the last cash flow
def FV(r, cf, continuous_compounding = False, dt = None, T = None):
    if dt is None: dt = range(len(cf))
    if T is None: T = dt[-1]
    fv = 0
    if not continuous_compounding:
        for k in range(0, len(cf)):
            fv += cf[k] * (1+r)**(T - dt[k])
    else:
        for k in range(0, len(cf)):
            fv += cf[k] * math.exp(r * (T - dt[k]))
    return fv   

# Computing the internal rate of return by using the fsolve function to find roots to a polynomial
def irr(cf):
    def simple_npv(r, cf):
        return sum(cf[k]/(1+r)**k for k in range(len(cf)))
    
    result = fsolve(simple_npv, x0 = 0.1, args = cf)
    return result[0]

# Computing the internal rate of return by using the irr function in numpy financial library
def irr2(cf):
    return npf.irr(cf)


**Problem 5**: (Tree cut) Suppose that you have the opportunity to plant trees that later can be sold for lumber. This project requires an initial outlay of money in order to purchase and plant the seedlings. No other cash flow occurs until the trees are harvested. However, you have a choice as to when to harvest. If you harvest after 1 year, you get your return quickly; but if you wait, the trees will have additional growth and the revenue generated from the sale of the trees will be greater. Assume that the cash flow streams associated with these alternatives are:
  - Wait 1 year: $(-1, 2)$
  - Wait 2 year: $(-1, 0, 3)$
  - Wait 3 year: $(-1, 0, 0, 4)$
  - $\vdots$
  - Wait n year $(-1, 0, 0,\ldots, n+1)$

The prevailing interest rate is 10%. When is the best time to cut the trees?

**Solution:**

**Problem 6:** - (Equivalent rates) What rate of interest (to five digits) is equivalent to 10\% yearly under
(a) monthly compounding?
(b) continuous compounding?

**Solution:**

**Problem 7:**(A prize) A major lottery advertises that it pays the winner 10 million. However, this prize money is paid at the rate of 500,000 each year (with the first payment being immediate) for a total of 20 payments. What is the present value of this prize at 10\% interest?

**Solution:**

**Problem 11:** (An appraisal) You are considering the purchase of a nice home. It is in every way perfect for you and in excellent condition, except for the roof. The roof has only 5 years of life remaining. A new roof would last 20 years, but would cost 20,000. The house is expected to last forever. Assuming that costs will remain constant and that the interest rate is 5\%, what value would you assign to the existing roof?

Hint: Consider the present value of change the roof now, and then every 20 years, and change the root in 5 year, and then every 20 years. The difference between their present value is the value of existing roof.

**Solution:**

**Problem 12:**(Oil depletion allowance) A wealthy investor spends 1 million to drill and develop an oil well that has estimated reserves of 200,000 barrels. The well is to be operated over 5 years, producing the estimated quantities shown in the following table. It is estimated that the oil will be sold for 20 per barrel. The net income is also shown. 

A depletion allowance, for tax purposes, can be computed in either of two ways each year: 22\% of gross revenue up to 50\% of net income before such deduction (option 1), or the investment cost of the product, equal in this case to the unit cost of the reserves, 5 per barrel (option 2). The allowance is deducted from the net income to determine the taxable income. The investor is in the 45\% tax bracket.

-  Complete Table 2.6 and show that the total depletion allowance exceeds the original investment.
-  Calculate the PV and the IRR for this investment. Assume an interest rate of 20\%.

In [67]:
raw_data = pd.read_csv("./Week_2_Problem_12.csv")
display(raw_data)

,Year,Barrel Produced,Gross Revenue,Net Income,Option 1,Option 2,Depletion Allowance,Taxable Income,Tax,After Tax Income
0,1,80000,1600000,1200000,352000.0,400000.0,400000.0,800000.0,360000.0,440000.0
1,2,70000,1400000,1000000,NaN,NaN,NaN,NaN,NaN,NaN
2,3,50000,1000000,500000,NaN,NaN,NaN,NaN,NaN,NaN
3,4,30000,600000,200000,NaN,NaN,NaN,NaN,NaN,NaN
4,5,10000,200000,50000,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
raw_data['Option 1'] = pd.DataFrame([raw_data['Gross Revenue'] * 0.2, raw_data['Net Income'] * 0.5]).min()
raw_data['Option 2'] = raw_data['Barrel Produced'] * 5
raw_data['Depletion Allowance'] = pd.DataFrame([raw_data['Option 1'], raw_data['Option 2']]).max()
raw_data['Taxable Income'] = raw_data['Net Income'] - raw_data['Depletion Allowance']
raw_data['Tax'] = raw_data['Taxable Income'] * 0.45
raw_data['After Tax Income'] = raw_data['Net Income'] - raw_data['Tax']
display(raw_data)

,Year,Barrel Produced,Gross Revenue,Net Income,Option 1,Option 2,Depletion Allowance,Taxable Income,Tax,After Tax Income
0,1,80000,1600000,1200000,320000.0,400000,400000.0,800000.0,360000.0,840000.0
1,2,70000,1400000,1000000,280000.0,350000,350000.0,650000.0,292500.0,707500.0
2,3,50000,1000000,500000,200000.0,250000,250000.0,250000.0,112500.0,387500.0
3,4,30000,600000,200000,100000.0,150000,150000.0,50000.0,22500.0,177500.0
4,5,10000,200000,50000,25000.0,50000,50000.0,0.0,0.0,50000.0
